In [1]:
import argparse
import os
import sys
import psutil
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
#from datetime import datetime
import math
from IPython.display import display
from multiprocessing import cpu_count,Pool 
import multiprocessing
from joblib import Parallel, delayed
from orderedset import OrderedSet
import datetime

In [2]:
long_event = pd.read_hdf('../../long_events_.h5',key='LN_300')

In [3]:
def distance(data,lat,lng,idx):
    fLat = np.radians(data.LocationLat)
    fLon = np.radians(data.LocationLng)
    sLat = np.radians(lat)
    sLon = np.radians(lng)
    R = 3958.7564 #mi
    #R = 6371000.0 #meters
    #R = 6371.0 #km
    
    dLon = sLon - fLon
    dLat = sLat - fLat
    a = np.sin(dLat/2.0)**2 + (np.cos(fLat) * np.cos(sLat) * np.power(np.sin(dLon/2.0), 2))
    
    c = 2.0 * np.arctan2(np.sqrt(a), np.sqrt(1.0 - a))
    #data['dis2event_'+str(idx)] =  
    return R * c
    #return data

In [4]:
def check(data,filepath,key):    
    counter=1
    drop_index=[]
    long_events_ = pd.read_hdf(filepath,key=key)
    for idx,long_event in data.iterrows():
                    collision = data[(long_events_.EventId != long_event.EventId ) & (long_events_.State ==  long_event.State)
                                & 
                                (((long_events_.StartTime >= long_event.StartTime) &
                                    (long_events_.EndTime <= long_event.EndTime))
                                    |
                                    ((long_events_.StartTime < long_event.StartTime) &
                                    (long_events_.EndTime > long_event.StartTime)) 
                                    |
                                    ((long_events_.StartTime < long_event.EndTime) &
                                    (long_events_.EndTime > long_event.EndTime))
                                    |
                                    ((long_events_.StartTime < long_event.StartTime) &
                                    (long_events_.EndTime > long_event.EndTime)))
                                ]

                    if long_event.Type == 'W':
                        collision = collision[collision.AirportCode == long_event.AirportCode]

                    elif long_event.Type != 'W':
                        collison_W = collision[(collision.Type=='W')&(collision.AirportCode == long_event.AirportCode)]
                        collision_T = collision[collision.Type!='W']
                        collision_T = collision_T.assign(distance=pd.Series(
                            distance(collision_T,long_event.LocationLat,long_event.LocationLng,idx)).values)
                        if 'distance' not in collision_T.columns:
                            print ('error')
                        collision_T = collision_T [(collision_T.distance <=raduis)]
                        collision = pd.concat([collison_W,collision_T])
                    else:
                        print ('error: event type ',long_event.Type)
                        raise()
                    if not collision.empty:
                        counter+=1
                        drop_index.append(idx)
                    
    return drop_index

In [5]:
class WithExtraArgs(object):
    def __init__(self, func, **args):
        self.func = func
        self.args = args
    def __call__(self, df):
        return self.func(df, **self.args)
def applyParallel_list(pool,data, func, kwargs):
    data_split = np.array_split(data,partitions)
    data_tag =pool.map(WithExtraArgs(func, **kwargs), data_split)
    return data_tag

def parallelize(data, func,pool):
    data_split = np.array_split(data, partitions)
    df = pool.map(func, data_split)
    return df

In [6]:
raduis = 14 # mi

cores = cpu_count() #Number of CPU cores on your system
partitions = cores #Define as many partitions as you want
print ("number of cores "+str(partitions))

number of cores 28


In [7]:
pool = Pool(cores)
drop =  applyParallel_list(pool,long_event,check,{'filepath': '../../long_events_.h5', 'key':'LN_300'})
pool.close()
pool.join()

/usr/local/anaconda5/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/anaconda5/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/anaconda5/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/anaconda5/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/anaconda5/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/anaconda5/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/anaconda5/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Boolea

In [8]:
print (len(drop))

28


In [9]:
temp = []
for l in drop:
    temp+=l
print (len(temp))
drop_ = temp

7916


In [10]:
long_event_drop1 = long_event.drop(drop_)

In [11]:
print (len(long_event)-len(long_event_drop1))

7916


In [13]:
long_event.loc[drop_[0]]

EventId         T-7438946_T-7438947_T-7438948_T-7439427_T-7439428
Type                                                            T
RefinedType_                                         Construction
StartTime                                     2018-05-15 13:51:48
EndTime                                       2018-05-31 04:59:00
LocationLat                                               34.7013
LocationLng                                              -86.6828
Distance(mi)                                                 8.77
AirportCode                                                  KHUA
Number                                                      30745
Street                                              US Highway 72
Side                                                            L
City                                                      Madison
County                                                  Limestone
State                                                          AL
ZipCode   

In [14]:
long_event.loc[drop_[1]]

EventId         T-7439431_T-7439432_T-7439433
Type                                        T
RefinedType_                     Construction
StartTime                 2018-05-15 14:02:50
EndTime                   2018-05-31 04:59:00
LocationLat                           34.6423
LocationLng                          -86.5325
Distance(mi)                             6.84
AirportCode                              KHUA
Number                                    NaN
Street                       Memorial Pkwy SW
Side                                        R
City                               Huntsville
County                                Madison
State                                      AL
ZipCode                                 35803
RefinedType                      Construction
duration                              22496.2
Name: 8281577, dtype: object

In [16]:
long_event_drop1.to_hdf('../../long_events_.h5',key='LN_300_nc1')

/usr/local/anaconda5/lib/python3.6/site-packages/pandas/core/generic.py:1299: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->['EventId', 'Type', 'RefinedType_', 'AirportCode', 'Street', 'Side', 'City', 'County', 'State', 'RefinedType']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [17]:
pool = Pool(cores)
drop = applyParallel_list(pool,long_event_drop1,check,{'filepath': '../../long_events_.h5', 'key':'LN_300_nc1'})
pool.close()
pool.join()

/usr/local/anaconda5/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/anaconda5/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/anaconda5/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/anaconda5/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/anaconda5/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/anaconda5/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/anaconda5/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Boolea

In [18]:
print (drop)

[[], [], [13634736, 13632572], [13034803, 13035373], [6794518, 5752887, 549357, 5462940, 9277706, 5454000, 549938, 5752062, 10780925, 10748864, 9278077, 3850037, 5463024, 5453986, 6890709], [], [], [], [], [3871141, 5744668, 9153925, 4498504, 746687, 4498859, 5745336, 5747126, 745838], [4430830, 9186224, 3277338, 1712866, 2108839, 1712888, 563215, 1183097, 2171719, 8456330, 3277820, 4426668, 3278374, 3266764, 3266979, 3277444, 3266095, 4426941, 3285054, 4430962, 8456343, 9214274], [], [], [], [], [], [], [11654123, 12770404, 10222206, 3380308, 790151, 7267095, 9317007, 4949304, 8653974, 5423339, 7605470, 9051505, 9082292, 1963946, 1028962, 1970112, 3382217, 7605353, 5424168, 10835508, 4949438, 9080050, 8678054, 9080049], [3501552, 3489821, 3509188, 3506765, 3509823, 3490663, 3501657, 5673863, 5673859, 3501769, 5673866, 3506932, 3510157, 3490854], [10132771, 13811270, 13715898, 15315585, 13846587, 13811535, 11590617], [9319963, 7183045, 6482703, 6572992, 3998799, 11631761, 10543875, 143

In [20]:
temp2 = []
for l in drop:
    temp2+=l
print (len(temp2))
drop_2 = temp2

214


In [29]:
con1=long_event_drop1.loc[drop_2[0]]

In [33]:
con2=long_event_drop1.loc[drop_2[1]]

In [34]:
if con2.StartTime < con1.StartTime and con2.EndTime > con1.StartTime:
    print ('yes')

yes


In [35]:
long_event_drop2 = long_event_drop1.drop(drop_2)

In [37]:
print (len(long_event_drop1)-len(long_event_drop2))

214


In [38]:
long_event_drop2.to_hdf('../../long_events_.h5',key='LN_300_nc2')

/usr/local/anaconda5/lib/python3.6/site-packages/pandas/core/generic.py:1299: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->['EventId', 'Type', 'RefinedType_', 'AirportCode', 'Street', 'Side', 'City', 'County', 'State', 'RefinedType']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [39]:
pool = Pool(cores)
drop = applyParallel_list(pool,long_event_drop2,check,{'filepath': '../../long_events_.h5', 'key':'LN_300_nc2'})
pool.close()
pool.join()

/usr/local/anaconda5/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/anaconda5/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/anaconda5/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/anaconda5/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/anaconda5/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/anaconda5/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/anaconda5/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Boolea

In [40]:
print (drop)

[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]


In [41]:
temp3 = []
for l in drop:
    temp3+=l
print (len(temp3))
drop_3 = temp3

0


In [42]:
In_LN = pd.read_hdf('../../long_events.h5',key='LN_300')

In [44]:
len(In_LN)

280649

In [43]:
data_set = pd.read_hdf('../../data_set.h5',key='DS_new')

In [45]:
len(data_set)

15325090

In [47]:

new_data_set = data_set.drop(In_LN.index)

In [48]:
len(new_data_set)

15044441

In [49]:
new_data_set = new_data_set.append(long_event_drop2)

In [50]:
len(new_data_set)

15192678

In [52]:
new_data_set.to_hdf('../../data_set_.h5',key='DS_300')

/usr/local/anaconda5/lib/python3.6/site-packages/pandas/core/generic.py:1299: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->['EventId', 'Type', 'RefinedType_', 'AirportCode', 'Street', 'Side', 'City', 'County', 'State', 'RefinedType']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [53]:
long_event_drop2.duration.max()

52435.01666666667

In [54]:
52435.01666666667/60/24

36.41320601851852

In [55]:
long_event_drop2.duration.min()

300.0

In [56]:
long_event_drop2.duration.mean()

926.9473225308171

In [58]:
new_data_set.index.get_duplicates()

[]